<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Image Processing Laboratory Notebooks</h2>
<hr style="clear:both">
<p style="font-size:0.85em; margin:2px; text-align:justify">
This Juypter notebook is part of a series of computer laboratories which are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course <b>Image Processing I</b> 
(<a href="https://moodle.epfl.ch/course/view.php?id=522">MICRO-511</a>) taught by Prof. M. Unser and Prof. D. Van de Ville.
</p>
<p style="font-size:0.85em; margin:2px; text-align:justify">
The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the <a href="http://bigwww.epfl.ch/">Biomedical Imaging Group</a>. 
The distribution or the reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2021.
</p>
<p style="font-size:0.85em; margin:0px"><b>Authors</b>: 
    <a href="mailto:pol.delaguilapla@epfl.ch">Pol del Aguila Pla</a>, 
    <a href="mailto:kay.lachler@epfl.ch">Kay Lächler</a>,
    <a href="mailto:alejandro.nogueronaramburu@epfl.ch">Alejandro Noguerón Arámburu</a>, and
    <a href="mailto:daniel.sage@epfl.ch">Daniel Sage</a>.
</p>
<hr style="clear:both">
<h1>Lab 1.1: Pixel-wise operations</h1>
<div style="background-color:#F0F0F0;padding:4px">
    <p style="margin:4px;"><b>Released</b>: Thursday October 7, 2021</p>
    <p style="margin:4px;"><b>Submission</b>: <span style="color:red">Friday October 15, 2021</span> (before 11:59PM) on <a href="https://moodle.epfl.ch/course/view.php?id=522">Moodle</a></p>
    <p style="margin:4px;"><b>Grade weigth</b>: Lab 1 (16 points), 9% of the overall grade</p>
    <p style="margin:4px;"><b>Remote help</b>: Monday October 11, on Zoom (see Moodle for link and time)</p>    
    <p style="margin:4px;"><b>Related lectures</b>: Chapters 1 and 2</p>
</div>

### Student Name: Guanqun LIU
### SCIPER: 334988

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

In [1]:
%use sos
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

SCIPER: 334988


### Imports
In the next cell we import the python libraries that we will use throughout the lab, as well as the `ImageViewer` class (Python package developed specifically for these laboratories, see documentation [here](https://github.com/Biomedical-Imaging-Group/interactive-kit/wiki/Image-Viewer), or run the python command `help(viewer)` after loading the class):
* [`matplotlib.pyplot`](https://matplotlib.org), to display images,
* [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/), to make the image display interactive, and
* [`numpy`](https://numpy.org/doc/stable/reference/), for mathematical operations on arrays.

Finally, we load the images used in this lab. Note that we will use the [OpenCV](https://opencv.org/) library for loading the images, and then we will make sure that it is of type `float64`, for higher accuracy during operations.

Run the next cell to get your notebook ready.
<div class=" alert alert-danger">
    
<i>Note:</i> Always run the two import cells below before starting to work on the notebook.    
</div>

In [2]:
%use sos
# Configure plotting as dynamic
%matplotlib widget

# Import required packages for this lab
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import cv2 as cv
from interactive_kit import imviewer as viewer

# Loading images
hrct = cv.imread("images/hrct.tif", cv.IMREAD_UNCHANGED).astype('float64')

In the following cell we import the JS `ImageAccess` class. You can find the documentation of the class [here](https://biomedical-imaging-group.github.io/image-access/).

In [3]:
%use javascript
// import image-access as Image
var Image = require('image-access')

# Pixel-wise operations (9 points)

In this laboratory you will learn the basics of pixel-wise image processing by performing simple operations -operations that apply at a pixel level-. This has many practical applications such as image normalization, colorization, and just to get information of an image. However, part of the goal of this laboratory is for you to feel comfortable with the process of accessing pixels of an image. Therefore, the laboratory will be mostrly in *JavaScript*, and then we will use *Python* for a few applications.

## <a id="ToC_1_Pixelwise_operations"></a>Table of contents
1. [16-bit gray-scale images: Visualization and Colorization](#1.-16-bit-gray-scale-images:-visualization-and-colorization-(3-points)) **(3 points)**
    1. [Limitations in visualization](#1.A.-Limitations-in-visualization-(1-point))
    2. [Colorization](#1.B.-Colorization-(2-points))
2. [Image normalization](#2.-Image-normalization-(6-points)) **(6 points)**
<!-- 3. [Vignetting](#3.-Vignettimg-(1-point)) -->

<div class=" alert alert-danger">
    
<b>Important:</b> Each cell that contains code begins with <code>%use sos</code> or <code>%use javascript</code>. This indicates if the code in this specific cell should be executed in Python or JavaScript. Do not change or remove any lines of code that begin with an <code>%</code>. They need to be on the first line of each cell!
    
</div>

# 1. 16-bit gray-scale images: visualization and colorization (3 points)
[Back to Table of contents](#ToC_1_Pixelwise_operations)

Most standard screens use only $8$ bits to visualize gray-level images, which allows for $2^8 = 256$ different gray-levels to be displayed at the same time. For day-to-day photography this is enough, but for some applications -like medical imaging- $256$ values are not enough to store all the important information. For example, common *HRCT* are stored in $16$-bit images which provide $2^{16} = 65536$ different gray-levels. This poses a problem because a standard screen cannot show all the gray-levels in a $16$-bit image at the same time. One solution for this limitation is to divide a range into sub-ranges of different colors (_a.k.a._ [colormaps](https://matplotlib.org/stable/tutorials/colors/colormaps.html)). There are many that have been proposed, and they serve the purpose of increasing the contrast on an image. In this section you will first see the limitation of displaying a $16$-bit grayscale image in a standard screen (in fact this goes beyond the screen, as your eyes are not capable of distinguising that many shades of gray either<!--only capable of distinguishing between $700$ and $900$ shades of gray-->), and then you will apply a colorization on this image.   

## 1.A. Limitations in visualization (1 point)
[Back to Table of contents](#ToC_1_Pixelwise_operations)

To illustrate the problem we will look at the image `hrct`, which is encoded with $16$ bits. The image shows the result of a [computed tomography](https://en.wikipedia.org/wiki/CT_scan) scan of a human thorax. These type of images can, for example, be used to diagnose or assess the developement of COVID-19 in patients ([see more here](https://radiologyassistant.nl/chest/covid-19/covid19-imaging-findings)). 

Run the next cell and explore the gray-level range. Try to find hidden content in the image that is not visible at first (at first you will only see the thorax). 
<div class="alert alert-info">
    
**Hint:** You can adjust the gray-level range of the image by adjusting the values (max / min) of the <i>Brightness & Contrast</i> slider. Take some time to properly explore the image, as well as the histogram. 
</div>

In [4]:
%use sos
# Display the hrct image to find hidden information
plt.close("all")
hrct_viewer = viewer(hrct, title = 'HRCT', hist=True, widgets=True)

### Multiple choice question
For **0.5 points each**, once you have explored the image, answer the next questions:

* Q1: How many $8$-bit grayscale images do we need to cover the pixel values spanned by a generic $16$-bit image?
    1. 2,
    2. 8,
    3. 256, or
    4. 16.


* Q2: As you will study below, using color is another option to see more of the information contained in a 16-bit image on screen. Now, choose `Options` and select the colormap `nipy_spectral` (make sure that the *Brightness & Contrast* slider spans the whole range). This view reveals wide-spread structures within the patient's lungs, which could be relevant to doctors. Which of the following ranges of visualization would show those details best with a `gray` colormap?
    1. $0\%$ to $10\%$,
    2. $90\%$ to $100\%$, or
    3. $50\%$ to $60\%$.

Modify the variables `answer_one` and `answer_two` in the next cell to reflect your choice.

In [5]:
%use sos
# Assign your answer to this variable
answer_one = 3
answer_two = 1
# YOUR CODE HERE

In [6]:
%use sos
# Check that the answer is in the correct range
assert answer_one in [1,2,3,4], 'Possible values are 1, 2, 3 or 4.'

In [7]:
%use sos
# Check that the answer is in the correct range
assert answer_two in [1,2,3], 'Possible values are 1, 2, or 3.'

## 1.B. Colorization (2 points) 
[Back to Table of contents](#ToC_1_Pixelwise_operations)

As we will see in this section, color is a powerful tool to overcome the representation limitations of $8$-bit screens, and make use of all your photoreceptors to see the information in an image. The basic idea is to express the $16$-bit range $[0, 65535]$ using a combination of three $8$-bit $[0, 255]$ channels to make a color (RGB) image. This exercise will guide you through this process.

There are many alternatives to divide a range into sub-ranges (again, take a look at the matplitlib [colormaps](https://matplotlib.org/stable/tutorials/colors/colormaps.html)), but you will implement the one specified by the picture below:

<img src="images/graylevel_divide_rgb.png" alt="Drawing" style="width: 500px;"/>

The three lines reflect the intensity values of each channel depending on the original graylevel intensity. $N$ is the maximum graylevel intensity for a 16-bit image, i.e., ($2^{16} - 1$).

For **1 point**, modify the function `color_pixel_wise(img)` in the next cell to iterate through every pixel in the image and create a new colorized image. The three color channels should be defined according to the figure above.
<div class="alert alert-info">

**Note:** You only need to modify the variables `r`, `g` and `b`. Everything else has already been prepared for you. Make sure you understand the code and fill in the blanks.
</div>

In [8]:
%use javascript
// function that divides a 16 bit graylevel image into an rgb image with 8 bits per channel
function color_pixel_wise(img){
    // the max value of the original image (16 bits)
    var N = Math.pow(2,16) - 1;
    // the max value of each channel of the new image (8 bits)
    var new_N = 255;
    // initialize output image (color image)
    options = {}; options.rgb = true;
    var output = new Image(img.ny, img.nx, options);
    // Iterate through each pixel
    for(var x = 0; x < img.nx; x++){
        for(var y = 0; y < img.ny; y++){
            // initialize the red, green and blue channels to 0
            var r = 0, g = 0, b = 0;
            // get the pixel value at the current location
            var value = img.getPixel(x, y);
            
            // assign the correct values to the red, green and blue channels according to the proposed mapping
            if (value < N/4){
                r = new_N * value / (N / 4);
                g = 0;
                b = 0;
                }
            
            else if ((N/4 < value) && (value < N/2)){
                r = 510 - (new_N * value / (N / 4));
                g = (new_N * value / (N / 4)) - 255;
                b = 0;
                }
            
            else if ((N/2 < value) && (value < 3*N/4)){
                r = 0;
                g = 765 - (new_N * value / (N / 4));
                b = (new_N * value / (N / 4)) - 510;
                }
            
            else if ((3*N/4 < value)){
                r = 0;
                g = 0;
                b = 1020 - (new_N * value / (N / 4));
                }
            
            // set the three color channels in the output image (convert them to integers using Math.round)
            output.setPixel(x, y, [Math.round(r), Math.round(g), Math.round(b)])
        }
    }
    return output;
}

Great, using the two cells below we are going to visualize your image. If everything went well, you should see most of the hidden details in red (blood vessels and details inside the lungs), the middle values in green (soft tissues, fat, etc.) and the higher values in blue (aorta, bone tissue, etc.). 

In the next cell, we are first going to get the image `hrct` from the Python kernel, and then apply your function to it. Finally, we put the result (`hrct_colorized_js`) back into the Python kernel for visualization. 

In [9]:
%use javascript
%get hrct
%put hrct_colorized_js

var hrct_img = new Image(hrct);
var hrct_colorized_js = color_pixel_wise(hrct_img).toArray();

Now that your result is stored in Python, run the next cell to visualize it.

<div class="alert alert-info">
    
<b>Note: </b> SoS translates JS arrays as Python lists. However, <code>ImageViewer</code> (and every major IP library) works with <i>NumPy</i> arrays, so we have to explicitly call the function <code>np.array</code> on the result from JS.
</div>

In [10]:
%use sos
# Convert to NumPy array
hrct_colorized_js = np.array(hrct_colorized_js)
# Visualize
plt.close('all')
hrct_colorized_viewer = viewer(hrct_colorized_js, title='HRTC Colorized in JS')

Button(description='Show Widgets', style=ButtonStyle())

Is your result correct? At the top of the image you should see the color bar transition **smoothly** from black to red, from red to green, from green to blue, and back to black again.

Let's make some sanity checks to test that the image is really a color image and that the maximum and minimum value of each channel are $0$ and $255$ respectively. We will constantly be making these kinds of checks to ensure the correct behaviour of any code you write.

<div class="alert alert-danger">

<b>Note:</b> The fact that you pass these sanity checks <b>does not</b> guarantee the points.
</div>

In [11]:
%use sos
# Check that the image has indeed 3 color channels
assert hrct_colorized_js.shape[2] == 3, "The resulting image doesn't have 3 color channels!"
# Check that the max and min of each channel are 0 and 255
assert np.min(hrct_colorized_js[:,:,0]) == 0, f"The minimum of the red color channel is {np.min(hrct_colorized_js[:,:,0])} and not 0!"
assert np.min(hrct_colorized_js[:,:,1]) == 0, f"The minimum of the green color channel is {np.min(hrct_colorized_js[:,:,1])} and not 0!"
assert np.min(hrct_colorized_js[:,:,2]) == 0, f"The minimum of the blue color channel is {np.min(hrct_colorized_js[:,:,2])} and not 0!"
assert np.max(hrct_colorized_js[:,:,0]) == 255, f"The maximum of the red color channel is {np.max(hrct_colorized_js[:,:,0])} and not 255!"
assert np.max(hrct_colorized_js[:,:,1]) == 255, f"The maximum of the green color channel is {np.max(hrct_colorized_js[:,:,1])} and not 255!"
assert np.max(hrct_colorized_js[:,:,2]) == 255, f"The maximum of the blue color channel is {np.max(hrct_colorized_js[:,:,2])} and not 255!"
print("Congrats, your function passed the sanity checks. However, that does not necessarily mean that everything is correct.")

Congrats, your function passed the sanity checks. However, that does not necessarily mean that everything is correct.


If we wanted to perform the same colorization in *Python*, we shouldn't do it pixel-wise as in *JavaScript*, since this will be extremely slow. As you saw in the introductory lab, languages that allow __[vectorization](https://en.wikipedia.org/wiki/Automatic_vectorization)__ like Python (through the NumPy library) and MATLAB allow operations to be performed on whole arrays. This is much simpler to code and as fast as a `for` loop in a low-level programming language. 

In the following cell, we show you how to colorize your image in Python, without the need to iterate through every pixel. This function is exactly like the one you created in JavaScript, but we added the parameters `peak1`, `peak2` and `peak3`, which will be useful in the next exercise. Make sure to fully understand the function. If you have any doubts, go back to the [Introductory lab](../Introductory_lab/Introductory.ipynb), or read about [indexing in NumPy](https://numpy.org/devdocs/reference/arrays.indexing.html).

When you run the next cell, you will generate and visualize the variable `hrct_colorized_python`. It should look exactly like the result you got from the JavaScript function above. 

In [12]:
%use sos
# Function that divides a `bits` bit graylevel image into a rgb image with 8 bits per channel depending on the three specified limits (in %)
def color_vectorized(img, peak1=25, peak2=50, peak3=75, bits=16):     
    # Initialize max value
    N = 2**bits - 1
    N_new = 2**8 - 1
    
    # Make sure that peak1 < peak2 < peak3
    peak1, peak2, peak3 = np.sort([peak1, peak2, peak3])
    
    # Adjust limit values from percent to absolute value
    peak1 = peak1 / 100 * N
    peak2 = peak2 / 100 * N
    peak3 = peak3 / 100 * N
    
    # Initialize 3 color channels with the appropriate dimensions
    color_R = np.zeros(img.shape)
    color_G = np.zeros(img.shape)
    color_B = np.zeros(img.shape)
    
    # Generate boolean arrays corresponding to the 4 different sections. 
    section_1 = img < peak1
    section_2 = np.logical_and(peak1 <= img, img < peak2)
    section_3 = np.logical_and(peak2 <= img, img < peak3)
    section_4 = peak3 <= img
    
    # Assign the pixel values of each channel depending on the section
    color_R[section_1] = img[section_1] / peak1
    color_R[section_2] = 1 - (img[section_2] - peak1) / (peak2 - peak1)
    color_G[section_2] = (img[section_2] - peak1) / (peak2 - peak1)
    color_G[section_3] = 1 - (img[section_3] - peak2) / (peak3 - peak2)
    color_B[section_3] = (img[section_3] - peak2) / (peak3 - peak2)
    color_B[section_4] = 1 - (img[section_4] - peak3) / (N - peak3)
    
    # Concatenate the three color channels into one color image    
    color_img = np.dstack((color_R,color_G,color_B))
    # Multiply by maximum and round to 8-bit integers
    color_img = np.round(color_img * N_new).astype(np.uint8)

    return(color_img)


# Run the function for the same (evenly spaced) limits as in the JavaScript function
hrct_colorized_python = color_vectorized(hrct)

# Visualize results
plt.close('all')
hrct_colorized_python_viewer = viewer([hrct_colorized_js, hrct_colorized_python], 
                                      title=['JS colorization', 'Python colorizaiton'], subplots=(1,2)) 

Button(description='Show Widgets', style=ButtonStyle())

Besides the visual test, you can use the next cell to compare your implementation to our function in Python. The results should be nearly identical. We use the function [`numpy.allclose`](https://numpy.org/doc/stable/reference/generated/numpy.allclose.html), and compare the two arrays. Run the next cell, if it runs smoothly, your implementation is correct! If not, we will use the comparison tool of the `viewer` to try to find your mistakes.  

In [13]:
%use sos

# Test if the two images are almost the same
if np.allclose(hrct_colorized_python, hrct_colorized_js):
    print('Congratulations! Your JS colorization seems to be perfect.')
else : 
    print('You still have a few errors! Hint: Check your range limits. ')
    viewer([hrct_colorized_python, hrct_colorized_js], compare=True)

Congratulations! Your JS colorization seems to be perfect.


As you can see, this is already a much better representation than only using one gray-level channel. However, we can do even better: In the above image, we used evenly spaced peaks for the triangles in the color mapping. However, if we know that some intensity ranges of the image contain more information than others, we can adjust the peaks to increase the visibility of these intensity ranges.

In the cell below we will add an extra functionality to the `viewer`: We will declare three sliders that will allow us to dynamically set the peaks of the triangles in our colorization, as well as the corresponding button and activation function (to review how this works, check the [introductory lab](../Introductory_lab/Introductory.ipynb#4.C.-User-defined-widgets)). The activation function will get the value of the sliders, and call the method `color_vectorized()` on the input image.

Below the viewer, we will plot a histogram of the image, overlayed with the triangles implemented by `color_vectorized()` for each slider selection. In this histogram the peaks will update in real time as you move the sliders in the viewer. This is meant for you to get a better idea of which ranges of values in the image enclose more information, and how one should design a colorization to reveal this information. **You do not need to understand this code-block**, however, if you are curious about `matplotlib` and `ipywidgets`, take the time to understand what we are doing.

Run the next cell and click on the button *Extra Widgets* to play with the three sliders. To show the effect of the current slider selection, click on `Apply Colorization`.

In [14]:
%use sos
plt.close('all')

# Defining the sliders and the button of the extra widget
peak1_slider = widgets.IntSlider(value = 25, min = 0, max = 100, step = 1, description = 'Peak 1 (%)')
peak2_slider = widgets.IntSlider(value = 50, min = 0, max = 100, step = 1, description = 'Peak 2 (%)')
peak3_slider = widgets.IntSlider(value = 75, min = 0, max = 100, step = 1, description = 'Peak 3 (%)')
activation_button = widgets.Button(description = 'Apply Colorization')

# Sort sliders whenever a user crosses them
def sort_sliders():
    peak1_slider.value, peak2_slider.value, peak3_slider.value = np.sort([peak1_slider.value, 
                                                                          peak2_slider.value, 
                                                                          peak3_slider.value])
# Defining the callback function of the button
def activation_callback(img):
    # Sort sliders (should not be necessary)
    sort_sliders()
    # Colorize image
    output = color_vectorized(img, peak1_slider.value, peak2_slider.value, peak3_slider.value)
    return output

# Visualize the image with the extra widget functionality
colorization_ranges_viewer = viewer(hrct, title = 'Personalizing your colormap', 
                                    new_widgets = [peak1_slider, peak2_slider, peak3_slider, activation_button], 
                                    callbacks = [activation_callback], widgets=True)

## The code below plots the interactive histogram.
## Feel free to explore it, but without any pressure.

# Maximum value in image
N = np.amax(hrct)
# Compute the histogram
hist, bins = np.histogram(hrct, bins = 70, range = (0, N))
# 10% over maximum number of counts (arbitrary max value for triangles)
Y = 1.1*np.amax(hist)
# Declare a matplotlib figure, capture its axes, plot the histogram, select axis' limits, and set x ticks to %
fig = plt.figure(num=f"SCIPER: {uid}",figsize = (4, 2.7)); ax = plt.gca()
ax.bar(bins[:-1], hist, width = (bins[1] - bins[0]) / 1.2)
ax.set_xlim(0, N); ax.set_ylim(0, Y); plt.yticks([],[])
plt.xticks([0,.25*N,.5*N,.75*N,N],[r"$0\%$",r"$25\%$",r"$50\%$",r"$75\%$",r"$100\%$"])
plt.title("Image Histogram and Colorization")
# List to store the lines that will form the triangles
lines = []
# Function that generates the triangles based on the peaks' positions
def generate_triangles(peak1, peak2, peak3):
    peak1, peak2, peak3 = np.sort([peak1, peak2, peak3])
    xdata = [[0, peak1], [peak1, peak2], [peak1, peak2], [peak2, peak3] , [peak2, peak3], [peak3, N]]
    ydata = [[0, Y], [Y, 0], [0, Y], [Y, 0] , [0, Y], [Y, 0]]
    return xdata, ydata

# Initial plot of the lines that form the triangles (2 per triangle)
color = 2*'r' + 2*'g'+ 2*'b' 
for i in range(6):
    xdata, ydata = generate_triangles(N/4, N/2, 3*N/4)
    lines.append(ax.plot(xdata[i], ydata[i], color[i]))
    
# Callback of sliders
def update_histogram(change):
    sort_sliders()  
    # Get the data 
    xdata, _ = generate_triangles(N*peak1_slider.value/100, N*peak2_slider.value/100, N*peak3_slider.value/100)
    # Update lines
    count = 0    
    for line in lines:
        line[0].set_xdata(xdata[count])
        count += 1
        
# Link sliders to callback (the three to the same callback)
for slider in [peak1_slider, peak2_slider, peak3_slider]:
    slider.observe(update_histogram, 'value') 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

### Multiple choice question
For **1 point**, which of the following combination of peaks allows you to see the most information?
<div class="alert alert-success">
    
<b>Tip:</b> If you do not see the sliders' values, collapse the file browser on the left and/or reduce the magnification of your browser.
</div>

1. `Peak 1` = $10\%$, `Peak 2` = $30\%$, `Peak 3` = $60\%$,
2. `Peak 1` = $25\%$, `Peak 2` = $50\%$, `Peak 3` = $75\%$, or
3. `Peak 1` = $10\%$, `Peak 2` = $60\%$, `Peak 3` = $85\%$.

Modify the variable `answer` in the next cell to reflect your answer.

In [15]:
%use sos
# Assign your answer to this variable
answer = 3
# YOUR CODE HERE

In [16]:
%use sos
# Check that the answer is in the valid range
assert answer in [1, 2, 3], 'Possible answers are 1, 2 or 3.'

# 2. Image normalization (6 points)
[Back to table of contents](#ToC_1_Pixelwise_operations)

In this section you will learn different ways to normalize an image.

**For a total of 3 points**, your assignment is to complete the three functions below (**1 point each**), which output images normalized with respect to different statistics. In particular, you have to complete:
* `makeZeroMean(img)`: Normalizes the image so that the sample mean of the pixel values is zero.
* `stretchContrast(img)`: Normalizes the image so that the minimum value is $0$ and the maximum value is $1$ (the contrast should be the same). 
* `normalize2ndOrderStatistics(img)`: Normalizes the image so that the sample mean of the pixel values is zero and the sample standard deviation is $1$. 

JS, unlike Python, **does not** have a function to calculate the mean or the standard deviation. Thus, you will need to code them yourself explicitly.
<div class="alert alert-success">
    
<b>Hint: </b> You can use <code>img.getMin()</code> and <code>img.getMax()</code> to get the min an max value of a JavaScript image <code>img</code>. Remeber also that you can access a wide range of mathematical functions through the Math library in JS (e.g., <code>Math.sqrt()</code>). You can read more about it [here](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Math).
</div>

First, implement the function `makeZeroMean` in the cell below.

In [17]:
%use javascript
// function that normalizes the image so that the sample mean of the pixel values is zero.
function makeZeroMean(img){
    // declare the output image
    var output = new Image(img.shape());
    
    // YOUR CODE HERE
    var pixel_sum = 0;
    var mean = 0;
    
    // Calculate the sum of pixel values across the image
    for (var x = 0; x < img.nx; x++){
        for (var y = 0; y < img.ny; y++){
            var pixel = img.getPixel(x, y);
            pixel_sum = pixel_sum + pixel;
            }
        }
    
    // Calculate the mean pixel value of the image
    mean = pixel_sum / (img.nx * img.ny);
    
    // For each pixel, normalize the value by subtracting the mean
    for (var x = 0; x < img.nx; x++){
        for (var y = 0; y < img.ny; y++){
            var normal_value = img.getPixel(x, y) - mean;
            output.setPixel(x, y, normal_value);
            }
        }
    // console.log(output);
    
    // return the output image
    return output;
}


Great! Now it's time to test your implementation. A partial test is to run the next cell, which will test your method on a simple $3\times 3$ array. If an error is thrown, you implementation is not yet correct.   

In [18]:
%use javascript

// declare the test image
test_img = new Image([[0, 1, 2], [3, 4, 5], [6, 7, 8]]);

// run the zero mean function
var test_zero_mean = makeZeroMean(test_img);

// check if the output is as expected 
if(!(test_zero_mean.imageCompare(new Image([[-4, -3, -2], [-1, 0, 1], [2, 3, 4]])))){
        console.log("makeZeroMean() is not yet correct");
}else{// print victory message
    console.log('Nice, the function seems to be correct!');}

Nice, the function seems to be correct!


Now, implement `stretchContrast`.

In [19]:
%use javascript

// function that normalizes the image so that all pixels have values between 0 and 1.
function stretchContrast(img){
    // declare the output image
    var output = new Image(img.shape());
    
    // YOUR CODE HERE
    // obtain min and max pixel values of the image
    var p_min = img.getMin();
    var p_max = img.getMax();
    
    // console.log(p_min);
    // console.log(p_max);
    
    // For each image pixel, normalize its pixel value to range (0, 1)
    for (var x = 0; x < img.nx; x++){
        for (var y = 0; y < img.ny; y++){
            var s_contrast = (img.getPixel(x, y) - p_min) / (p_max - p_min);
            // set the output image to the normalized scale
            output.setPixel(x, y, s_contrast);
            }
        }
    
    // return the output image
    return output;
}

Run the next cell to test the function.

In [20]:
%use javascript

// run the stretch contrast function on the test image
var test_stretch = stretchContrast(test_img);

// compare the result to the correct result
if(!(test_stretch.imageCompare(new Image([[0, 0.125, 0.25], [0.375, 0.5, 0.625], [0.75, 0.875, 1]])))){
    console.log("stretchContrast() is not yet correct.");
}else{
    console.log('Nice, the function seems to be correct!');}

Nice, the function seems to be correct!


Finally, implement the function `normalize2ndOrderStatistics`.

In [21]:
%use javascript

// function that normalizes the image so that the sample mean of the pixel values is 0 and the sample standard deviation is 1.
function normalize2ndOrderStatistics(img){
    // declare the output image
    var output = new Image(img.shape());
    
    // YOUR CODE HERE
    var pixel_sum = 0;
    var mean = 0;
    var diff_sum = 0;
    var sd = 0;
    
    // Calculate the sum of pixel values across the image
    for (var x = 0; x < img.nx; x++){
        for (var y = 0; y < img.ny; y++){
            var pixel = img.getPixel(x, y);
            pixel_sum = pixel_sum + pixel;
            }
        }
    
    // Calculate the mean pixel value of the image
    mean = pixel_sum / (img.nx * img.ny);
    // console.log(mean)
    
    // For each pixel, normalize the value by subtracting the mean
    // Now the output image is normalized to sample mean 0
    for (var x = 0; x < img.nx; x++){
        for (var y = 0; y < img.ny; y++){
            var normal_value = img.getPixel(x, y) - mean;
            diff_sum = diff_sum + Math.pow(normal_value, 2);
            output.setPixel(x, y, normal_value);
            }
        }
    
    // console.log(output)
    // console.log(diff_sum)
    
    // Calculate the standard deviation of all pixels in the image
    sd = Math.sqrt(diff_sum / (img.nx * img.ny));
    // console.log(sd)
    
    // Normalize the standard deviation to 1
    for (var x = 0; x < img.nx; x++){
        for (var y = 0; y < img.ny; y++){
            var normal_sd = output.getPixel(x, y) / sd;
            output.setPixel(x, y, normal_sd);
            }
        }
    
    console.log(output)
    // return the output image
    return output;
}

And run the next cell for a quick test again.

In [22]:
%use javascript

// run the function on test_img
var test_normalize = normalize2ndOrderStatistics(test_img).toArray();

// compare the result to the correct result
// test for unbiased estimator of variance
if(!(Image.arrayCompare(test_normalize, [[ -1.4605934866804429, -1.0954451150103321, -0.7302967433402214 ],
                                         [ -0.3651483716701107, 0, 0.3651483716701107 ],
                                         [ 0.7302967433402214, 1.0954451150103321, 1.4605934866804429 ]]))){
    // test for biased estimator of variance
    if(!(Image.arrayCompare(test_normalize, [[ -1.5491933384829668, -1.161895003862225, -0.7745966692414834 ],
                                             [ -0.3872983346207417, 0, 0.3872983346207417 ],
                                             [ 0.7745966692414834, 1.161895003862225, 1.5491933384829668 ]]))){
        console.log("normalize2ndOrderStatistics() is not yet correct!");
    }else{
        // print victory message
        console.log('Nice, the function seems to be correct! You\'re using the biased estimator of the variance.');
    }
}else{
    // print victory message
    console.log('Nice, the function seems to be correct! You\'re using the unbiased estimator of the variance.');
}

ImageAccess {
  image: [
    [ -1.5491933384829668, -1.161895003862225, -0.7745966692414834 ],
    [ -0.3872983346207417, 0, 0.3872983346207417 ],
    [ 0.7745966692414834, 1.161895003862225, 1.5491933384829668 ]
  ],
  nx: 3,
  ny: 3
}
Nice, the function seems to be correct! You're using the biased estimator of the variance.


In order for you to see the relevance of image normalization, we provide a sequence of fluorescence microscopy images (see more [here](https://en.wikipedia.org/wiki/Fluorescence_microscope)), named `c_elegans`. These are consecutive slices of the same 3D volume, but appear darker every time due to photobleaching (the loss of flourescence, read more [here](https://en.wikipedia.org/wiki/Photobleaching)). Of course, this is a huge problem for the application, and to solve it we absolutely need to normalize these images.

Run the next cell to load the `c_elegans` series, and display it. We will use the module [io](https://scikit-image.org/docs/0.8.0/api/skimage.io.html) of SciKit-Image, which allows us to read all the slices of a `.tif` file at once. Furthermore, we will pass it to JS so that we can use the functions you defined above.

<div class = ' alert alert-success'>
<b>Hint</b>: In the next cell, you will see the original images, and a graph that shows the effect of photobleaching on the mean value of each image. Make sure you explore the images and their histogram by clicking on the <code>Prev</code> and <code>Next</code> buttons! And look at the histograms of the images.
</div>

In [23]:
%use sos
%put c_elegans --to javascript

# We import module io to import tif images as slices
from skimage import io
# The following cell loads the image c-elegans
c_elegans = io.imread( "images/c-elegans.tif" ) 
# Show c_elegans images\
plt.close("all")
viewer([c_elegans[ind,:,:] for ind in range(12)], normalize=False, clip_range = [0, 255], title=[f"c_elegans {ind+1}" for ind in range(12)], hist=True)
# Show decay of the mean value through time due to photobleaching
fig = plt.figure(num=f"SCIPER: {uid}",figsize = (6, 4))
plt.plot([ind+1 for ind in range(12)], [np.mean(c_elegans[ind,:,:]) for ind in range(12)])
plt.xticks([ind+1 for ind in range(12)]); plt.xlabel("Image number"); plt.ylabel("Mean value"); 
plt.grid('both'); plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

Now we are going to visualize the effect of each of your normalizing functions on the `c_elegans` images. For this purpose, we provide you the function `make_montage(img_arr, mode, cols)`, which takes as parameters:
* `img_arr`: an array of `Image` objects, 
* `mode`: the function to apply (1: zero mean, 2: stretch contrast, 3: normalize statistics), and
* `cols`: the number of columns to use in the montage. 

The function first creates an empy montage `out`. `out` has the dimensions to fit all the images given in `img_arr`. The method then performs the operation specified by `mode` on each image, placing the result in the right place inside the montage `out`. 

Now run the next cell to declare the function `makeMontage()`.

In [24]:
%use javascript

// function that creates a single image from multiple images (slices) and performs the specified function on the images
function makeMontage(img_arr, mode, cols) {
    // get dimensions of each image
    var w = img_arr[0].nx;
    var h = img_arr[0].ny;
    // determine the number of rows 
    var rows = img_arr.length/cols;
    // initialize output image 
    var out = new Image(h*rows, w*cols);    
    // iterate through each image in img_arr
    for(t=0; t<rows*cols; t++){
        // extract the corresponding image 
        var img = img_arr[t].copy();
        // check requested operation 
        //(note that any mode other than 1, 2 or 3 simply copies the original images in the montage)
        if(mode == 1){ 
            img = makeZeroMean(img);
        }             
        if(mode == 2){
            img = stretchContrast(img);
        }
        if(mode == 3){
            img = normalize2ndOrderStatistics(img);
        }            
        // put result in the corresponding place
        out.putSubImage(Math.floor(t%cols)*w, Math.floor(t/cols)*h, img);
    }
    // return output image
    return out;
}

Now, we are going apply your methods to the image slices we just loaded. First, we convert each element in the array `c_elegans`  to an `Image` object. Then, we call the function on the array with each of the three modes to visualize the result of the functions that you implemented above.

In [25]:
%use javascript
%put montage_original montage_zero_mean_js montage_normalize_statistics_js montage_stretch_contrast_js 

// convert each element in the c_elegans array to an Image object
var c_elegans_imgs = new Array();
for(x = 0; x < c_elegans.length; x++){
    c_elegans_imgs.push(new Image(c_elegans[x]));
}

// run makeMontage with all four functions (modes)
var montage_original = makeMontage(c_elegans_imgs, 0, 3).toArray();
var montage_zero_mean_js = makeMontage(c_elegans_imgs, 1, 3).toArray();
var montage_stretch_contrast_js = makeMontage(c_elegans_imgs, 2, 3).toArray();
var montage_normalize_statistics_js = makeMontage(c_elegans_imgs, 3, 3).toArray();

ImageAccess {
  image: [
    [
      -1.4233283743018112,  -1.340903012309482, -1.2859527709812626,
      -1.2859527709812626,  -1.258477650317153, -1.0936269263324947,
       -1.258477650317153, -1.2859527709812626,  -1.258477650317153,
      -1.2310025296530434, -1.4782786156300307, -1.3683781329735918,
       -1.258477650317153,  -1.340903012309482, -1.4233283743018112,
       -1.258477650317153, -1.5607039776223597,  -1.340903012309482,
      -1.2859527709812626,  -1.340903012309482, -1.4233283743018112,
       -1.258477650317153, -1.3134278916453723,  -1.450803494965921,
      -1.3134278916453723,  -1.340903012309482,  -1.258477650317153,
      -1.2859527709812626, -1.3683781329735918, -1.3134278916453723,
      -1.1211020469966044,  -1.258477650317153, -1.0386766850042755,
      -1.1211020469966044, -1.0936269263324947, -1.1211020469966044,
      -1.1485771676607142, -1.0936269263324947, -1.3134278916453723,
      -1.2310025296530434,  -1.176052288324824, -1.2310025296530434,
   

12

Now that we have applied your functions and that we have the variables in Python, let's visualize them. Run the following cell to do so. Use the buttons `Next` and `Prev` to browse through the tree images. If your implementations passed the previous tests, you should see the correct result.

In [26]:
%use sos
# Define the lists of images and titles
image_list = [np.array(montage_original), np.array(montage_zero_mean_js), np.array(montage_stretch_contrast_js), np.array(montage_normalize_statistics_js)]
title_list = ['Original c_elegans', 'Zero Mean c_elegans', 'Stretch Contrast c_elegans', 'Normalize Statistics c_elegans']

# Display the montages
plt.close('all')
normalization_viewer = viewer(image_list, title = title_list, widgets=True)

### Multiple choice questions

Why does the bottom-right corner of the **Zero Mean Montage** have lower contrast than the top-left corner? (**0.5 points**)

1. Because the different subimages in the montage have different spreads of values around their mean.
2. It is an illumination effect.
3. The bottom-right subimages of the montage are defective.

Modify the variable `answer` in the next cell to reflect your choices. As usual, there is another cell that will remind you to select a valid choice if you haven't.

In [27]:
%use sos
# Assign your answer to this variable
answer = 1
# YOUR CODE HERE

In [28]:
%use sos
assert answer in [1, 2, 3], 'Possible answers are 1, 2 or 3' 

Why is this not the case for the other two montages? (**0.5 points**)

1. Becuase the images already had zero mean.
2. Because the other two functions modify the contrast by adjusting the range of intensities.

Modify the variable `answer` in the next cell to reflect your choices. As usual, there is another cell that will remind you to select a valid choice if you haven't.

In [29]:
%use sos
# assign your answer to this variable
answer = 2
# YOUR CODE HERE

In [30]:
%use sos
assert answer in [1, 2], 'Possible answers are 1 and 2'

As you have probably realized by now, low level languages can get tedious. You can do the same thing you did in JavaScript in less lines in Python, by using NumPy arrays, so let's do it! 

We give you the method `make_zero_mean`. **For a total of 2 points**, implement the other two methods (`stretch_contrast` and `normalize_2nd_order_statistics`) in Python in the following cell (**1 point each**). When appropriate, use the following functions:
- `np.mean(img)` returns the estimated mean value of `img`,
- `np.min(img)` and `np.max(img)` return the min and max of `img` respectively,
- `np.std(img)` returns the estimated standard deviation of `img`, based on the biased estimator of the variance (see more about the `ddof` parameter running `help(np.std)`).

<div class="alert alert-info">

<b>Hint:</b><ul> 
<li>If you're unsure how to handle numpy arrays, look at Section 2.A.a of the <a href = "./Introductory_lab/Introductory.ipynb">Introductory lab</a> ),</li>
<li>Only one line of code needs to be filled in for every function, but do not worry if you prefer to use more lines.</li>
</ul></div>

In [55]:
%use sos
# Function that normalizes the image so that the sample mean of the pixel values is zero.
def make_zero_mean(img):
    # Declare the output image
    output = np.copy(img)
    # Subtract the mean from the input image
    output = img-np.mean(img)
    # Return the output image
    return output 

# Function that normalizes the image so that all pixels have values between 0 and 1.
def stretch_contrast(img):
    # Declare the output image
    output = np.copy(img)
    
    # Get no. of rows and columns for iteration
    rows = img.shape[0]
    columns = img.shape[1]
    
    # Calculate the range of pixel intensity
    p_range = np.max(img) - np.min(img)
    
    # For each pixel, rescale the pixel values to (0, 1)
    for x in range(rows):
        for y in range(columns):
            output[x][y] = (output[x][y] - np.min(img)) / p_range
    
    # Return the output image
    return output

# Function that normalizes the image so that the sample mean of the pixel values is 0 and the sample standard deviation is 1.
def normalize_2nd_order_statistics(img):
    # Declare the output image
    output = np.copy(img)
    
    # Normalize to N(0, 1) by output = (img - mean) / sd
    output = (img - np.mean(img)) / np.std(img)
    
    # Return the output image
    return output

Use the next two cells for a quick test on your functions. This cell tests the two characteristics requested for each function:
* that the result of `stretch_contrast` is in the range $[0, 1]$, and
* that the result of `normalize_2nd_order_statistics` has zero mean and unit variance. 

Run them, and if your implementations are correct, they shouldn't raise any errors.

In [56]:
%use sos
# This cell tests your method stretch contrast
# Here we run your function on the first slice of c_elegans
test_stretch_contrast = stretch_contrast(c_elegans[0])

# And we check that stretch_contrast effectively maps the pixels to the range [0,1]
assert np.min(test_stretch_contrast) == 0, 'The minimum value in the result of stretch_contrast is not 0'
assert np.max(test_stretch_contrast) == 1, 'The maximum value in the result of stretch_contrast is not 1'
print("Well done! Your stretch_contrast function seems to be correct.")

Well done! Your stretch_contrast function seems to be correct.


In [57]:
%use sos

# This cell tests your method normalize statistics
# Here we run the method on the first slice of c_elegans
test_normalize_statistics = normalize_2nd_order_statistics(c_elegans[0])

# Now we check that normalize_statistics returns an image with mean = 0, 
assert np.abs(np.mean(test_normalize_statistics)) < 1e-10, 'Your mean in normalize_2nd_order_statistics is not 0'
# And with std = 1 
assert np.abs(np.std(test_normalize_statistics) - 1) < 1e-4 or np.abs(np.std(test_normalize_statistics, ddof=1) - 1) < 1e-5, 'Your standard deviation in normalize_2nd_order_statistics is not 1'
print('Well done! Your normalization of 2nd order statistics seems to be correct.')

Well done! Your normalization of 2nd order statistics seems to be correct.


<div class="alert alert-success">
    
<p><b>Congratulations on finishing the first part of the Pixel-Fourier lab!</b></p>
<p>
Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to <a href="https://moodle.epfl.ch/mod/assign/view.php?id=1157357">Moodle</a>, in a zip file with the other notebook of this lab.
</p>
</div>

* Keep the name of the notebook as: *1_Pixelwise_operations.ipynb*,
* Name the zip file: *Pixel_Fourier_Lab.zip*.

<!-- <div class="alert alert-danger">
<h4>Feedback</h4>
    <p style="margin:4px;">
    This is the first edition of the image-processing laboratories using Jupyter Notebooks running on Noto. Do not leave before giving us your <a href="https://moodle.epfl.ch/mod/feedback/view.php?id=1157363">feedback here!</a></p>
</div> -->